<a href="https://colab.research.google.com/github/AshSama12/Anomaly-detection-in-wearing-PPE/blob/master/Copy_of_combine_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive. mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# File paths to datasets
ppe_dataset_path = '/content/drive/MyDrive/research2025/PPE_Annotation.v1i.yolov8-obb.zip'
human_dataset_path = '/content/drive/MyDrive/research2025/human detection new 2.v5i.yolov8-obb.zip'
output_dir_ppe = '/content/PPE_Annotation'
output_dir_human = '/content/Human_Annotation'

# Unzip the datasets
for zip_path, out_dir in [(ppe_dataset_path, output_dir_ppe), (human_dataset_path, output_dir_human)]:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(out_dir)
    print(f"Extracted {zip_path} to {out_dir}")


Extracted /content/drive/MyDrive/research2025/PPE_Annotation.v1i.yolov8-obb.zip to /content/PPE_Annotation
Extracted /content/drive/MyDrive/research2025/human detection new 2.v5i.yolov8-obb.zip to /content/Human_Annotation


In [ ]:
import shutil
import os

# Create the directories for the merged dataset
os.makedirs('/content/Merged_Dataset/train/images', exist_ok=True)
os.makedirs('/content/Merged_Dataset/train/labels', exist_ok=True)
os.makedirs('/content/Merged_Dataset/val/images', exist_ok=True)
os.makedirs('/content/Merged_Dataset/val/labels', exist_ok=True)
os.makedirs('/content/Merged_Dataset/test/images', exist_ok=True)
os.makedirs('/content/Merged_Dataset/test/labels', exist_ok=True)

# Define paths
human_train_img = '/content/Human_Annotation/train/images'
human_train_lbl = '/content/Human_Annotation/train/labels'
ppe_train_img = '/content/PPE_Annotation/train/images'
ppe_train_lbl = '/content/PPE_Annotation/train/labels'
merged_train_img = '/content/Merged_Dataset/train/images'
merged_train_lbl = '/content/Merged_Dataset/train/labels'

# Copy data
for src, dest in [
    (human_train_img, merged_train_img),
    (ppe_train_img, merged_train_img),
    (human_train_lbl, merged_train_lbl),
    (ppe_train_lbl, merged_train_lbl)
]:
    # Ensure the source directory exists
    if os.path.exists(src):
        for file_name in os.listdir(src):
            shutil.copy(os.path.join(src, file_name), dest)
    else:
        print(f"Source directory does not exist: {src}")

print("Data merged successfully!")


Data merged successfully!


In [ ]:
yaml_content = """
train: /content/Merged_Dataset/train/images
val: /content/Merged_Dataset/val/images
test: /content/Merged_Dataset/test/images

names:
  0: person
  1: helmet
  2: vest
  3: boots
"""
with open('/content/Merged_Dataset/data.yaml', 'w') as yaml_file:
    yaml_file.write(yaml_content)

print("data.yaml created successfully!")


data.yaml created successfully!


In [ ]:
pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.6/911.6 kB 22.4 MB/s eta 0:00:00


In [ ]:
import os

def clean_labels(label_dir, valid_classes):
    for label_file in os.listdir(label_dir):
        label_path = os.path.join(label_dir, label_file)
        with open(label_path, 'r') as f:
            lines = f.readlines()
        cleaned_lines = [line for line in lines if int(line.split()[0]) in valid_classes]
        if cleaned_lines:
            with open(label_path, 'w') as f:
                f.writelines(cleaned_lines)
        else:

            os.remove(label_path)

# Define paths
train_label_dir = "/content/Merged_Dataset/train/labels"
val_label_dir = "/content/Merged_Dataset/val/labels"
test_label_dir = "/content/Merged_Dataset/test/labels"

# Clean label directories
for label_dir in [train_label_dir, val_label_dir, test_label_dir]:
    clean_labels(label_dir, valid_classes={0, 1, 2, 3})

print("Label cleaning complete!")


Label cleaning complete!


In [ ]:
from PIL import Image

def validate_images(image_dir):
    for image_file in os.listdir(image_dir):
        image_path = os.path.join(image_dir, image_file)
        try:
            with Image.open(image_path) as img:
                img.verify()  # Verify if the image is not corrupted
        except Exception:
            print(f"Removing corrupted image: {image_path}")
            os.remove(image_path)

# Validate image directories
for image_dir in ["/content/Merged_Dataset/train/images", "/content/Merged_Dataset/val/images", "/content/Merged_Dataset/test/images"]:
    validate_images(image_dir)

print("Image validation complete!")


Image validation complete!


In [ ]:
import os

val_images_path = '/content/Merged_Dataset/val/images'
val_labels_path = '/content/Merged_Dataset/val/labels'

assert os.path.exists(val_images_path), "Validation images path does not exist!"
assert os.path.exists(val_labels_path), "Validation labels path does not exist!"

image_files = [f for f in os.listdir(val_images_path) if f.endswith(('jpg', 'jpeg', 'png', 'bmp', 'tiff'))]
label_files = [f for f in os.listdir(val_labels_path) if f.endswith('.txt')]

print(f"Found {len(image_files)} validation images.")
print(f"Found {len(label_files)} validation labels.")


Found 0 validation images.
Found 0 validation labels.


In [ ]:
import os

dataset_path = '/content/Merged_Dataset'
print(f"Available directories: {os.listdir(dataset_path)}")


Available directories: ['data.yaml', 'test', 'val', 'train']


In [ ]:
import os

# Define paths based on your structure
base_path = '/content/Merged_Dataset'
train_images_path = os.path.join(base_path, 'train', 'images')
train_labels_path = os.path.join(base_path, 'train', 'labels')
val_images_path = os.path.join(base_path, 'val', 'images')
val_labels_path = os.path.join(base_path, 'val', 'labels')

# Check files
train_images = sorted([f for f in os.listdir(train_images_path) if f.endswith(('jpg', 'jpeg', 'png'))])
train_labels = sorted([f for f in os.listdir(train_labels_path) if f.endswith('.txt')])

val_images = sorted([f for f in os.listdir(val_images_path) if f.endswith(('jpg', 'jpeg', 'png'))])
val_labels = sorted([f for f in os.listdir(val_labels_path) if f.endswith('.txt')])

print(f"Training images: {len(train_images)}")
print(f"Training labels: {len(train_labels)}")
print(f"Validation images: {len(val_images)}")
print(f"Validation labels: {len(val_labels)}")


Training images: 2796
Training labels: 2796
Validation images: 0
Validation labels: 0


In [ ]:
val_images = sorted([f for f in os.listdir(val_images_path) if f.endswith(('jpg', 'jpeg', 'png'))])
val_labels = sorted([f for f in os.listdir(val_labels_path) if f.endswith('.txt')])

print(f"Validation images: {len(val_images)}")
print(f"Validation labels: {len(val_labels)}")


Validation images: 0
Validation labels: 0
